In [1]:
from hamiltonian import HamiltonianSmall, Hamiltonian

import var_opt_lagrange
import time

# Lagrange by Rudy

See Rudy's note on optimizing the variance with Lagrange multiplier method

This notebook works for both convex cost function `diagonal` and non-convex cost function `mixed`

For the non-convex setting, we need access to the Hartree-Fock bitstrings. Currently, we only have that for JW encodings.

In [2]:
lih_small = HamiltonianSmall('LiH', 1.5) # this encoding uses reduction techniques
beh2_small = HamiltonianSmall('BeH2', 1.3) # this encoding uses reduction techniques

# 4 qubits
h2_jw_4 = Hamiltonian('H2_STO3g_4qubits', 'jw')
h2_parity_4 = Hamiltonian('H2_STO3g_4qubits', 'parity')
h2_bk_4 = Hamiltonian('H2_STO3g_4qubits', 'bk')

# 8 qubits
h2_jw = Hamiltonian('H2_6-31G_8qubits', 'jw')
h2_parity = Hamiltonian('H2_6-31G_8qubits', 'parity')
h2_bk = Hamiltonian('H2_6-31G_8qubits', 'bk')

# 12 qubits
lih_jw = Hamiltonian('LiH_STO3g_12qubits', 'jw')

# 14 qubits
h2o_jw = Hamiltonian('H2O_STO3g_14qubits', 'jw')

beh2_jw = Hamiltonian('BeH2_STO3g_14qubits', 'jw')

# 16 qubits
nh3_jw = Hamiltonian('NH3_STO3g_16qubits', 'jw')

# 20 qubits
c2_jw = Hamiltonian('C2_STO3g_20qubits', 'jw')

In [3]:
hamiltonians = {"lih_small": lih_small,
                "beh2_small": beh2_small,
                "h2_jw_4": h2_jw_4,
                "h2_parity_4": h2_parity_4,
                "h2_bk_4": h2_bk_4,
                "h2_jw": h2_jw,
                "h2_parity": h2_parity,
                "h2_bk": h2_bk,
                "lih_jw": lih_jw,
                "h2o_jw": h2o_jw,
                "beh2_jw": beh2_jw,
                "nh3_jw": nh3_jw,
                "c2_jw": c2_jw}

In [4]:
method = 'lagrange'

An example showing how to call this from `Hamiltonian.PauliRep.local_dists_optimal`

In [5]:
from numpy.random import rand
import numpy as np

np.random.seed(10)

def isProbability(betas, tol=1.0e-5):
    """
        Check if betas are probabilities
    """
    for k,v in betas.items():
        for _v in v:
            if _v < 0:
                return False
        if np.abs(np.sum(v) - 1.0) > tol:
            print("\tErr:", np.abs(np.sum(v) - 1.0) )
            return False
    return True

def get_rand_betas(num_qubits):
    """
        generate random initial points to search for betas
    """
    betas = {}
    for qubit in range(num_qubits):
        betas[qubit] = rand(3)
        betas[qubit] = betas[qubit]/np.sum(betas[qubit])
    return betas

In [6]:
def both_variances_lagrange(ham, num_trials=3, display_betas=True):
    t0 = time.time()
    energy, state = ham.pauli_rep.ground(multithread=True) # you can add optional keyword num_cores=15
    t1 = time.time()
    # diagonal objective function
    
    objective = 'diagonal'
    β_optimal_diagonal = ham.pauli_rep.local_dists_optimal(objective, method)
    if display_betas:
        print("\tbeta (diagonal):")
        for qubit in range(len(β_optimal_diagonal)):
            print("\t\t", qubit, β_optimal_diagonal[qubit])
    t2 = time.time()
    var = ham.pauli_rep.variance_local(energy, state, β_optimal_diagonal)
    t3 = time.time()
    print("Diagonal variance:", var)
    print("\n")
    
    # mixed objective function
    
    objective = 'mixed'
    assert ham.encoding == 'jw' # we do not have HF bitstrings for other encoding at the moment.
    bitstring_HF = ham.read_bitstring_HF()
    
    for trial in range(num_trials): 
        #because we do not have convexity, we perform several trials to look for optimal betas
        print("Trial:", _)
        if trial == 0:
            beta_init = β_optimal_diagonal
        else:
            beta_init = get_rand_betas(ham.pauli_rep.num_qubits)
        
        if trial == num_trials-1:
            t4 = time.time()
        
        β_optimal_mixed = ham.pauli_rep.local_dists_optimal(objective, method, 
                                                            bitstring_HF=bitstring_HF, β_initial=beta_init)
        
        if not isProbability(β_optimal_mixed):
            print("WARNING: not probability:", β_optimal_mixed)
        if display_betas:
            print("\tbeta (mixed):")
            for qubit in range(len(β_optimal_diagonal)):
                print("\t\t", qubit, β_optimal_diagonal[qubit])
        
        if trial == num_trials-1:
            t5 = time.time()
            
        var = ham.pauli_rep.variance_local(energy, state, β_optimal_mixed)
        
        if trial == num_trials-1:
            t6 = time.time()
        
        print("Mixed variance:", var)
        
        print("\n")
        
    print("Timings:")
    print("ground state: {} seconds".format(int(t1-t0)))
    print("optimal diagonal beta: {} seconds".format(int(t2-t1)))
    print("calculate variance: {} seconds".format(int(t3-t2)))
    print("optimal mixed beta (last trial): {} seconds".format(int(t5-t4)))
    print("calculate variance (last trial): {} seconds".format(int(t6-t5)))    

In [7]:
both_variances_lagrange(h2_jw_4)

	beta (diagonal):
		 0 [0.30214128 0.30214128 0.39571744]
		 1 [0.31122347 0.31122347 0.37755306]
		 2 [0.30214128 0.30214128 0.39571744]
		 3 [0.31122347 0.31122347 0.37755306]
Diagonal variance: 1.8555830498849535


Trial: 
	beta (mixed):
		 0 [0.30214128 0.30214128 0.39571744]
		 1 [0.31122347 0.31122347 0.37755306]
		 2 [0.30214128 0.30214128 0.39571744]
		 3 [0.31122347 0.31122347 0.37755306]
Mixed variance: 1.8546562200043255


Trial: 
	beta (mixed):
		 0 [0.30214128 0.30214128 0.39571744]
		 1 [0.31122347 0.31122347 0.37755306]
		 2 [0.30214128 0.30214128 0.39571744]
		 3 [0.31122347 0.31122347 0.37755306]
Mixed variance: 1.854656258586077


Trial: 
	beta (mixed):
		 0 [0.30214128 0.30214128 0.39571744]
		 1 [0.31122347 0.31122347 0.37755306]
		 2 [0.30214128 0.30214128 0.39571744]
		 3 [0.31122347 0.31122347 0.37755306]
Mixed variance: 1.854656259066888


Timings:
ground state: 0 seconds
optimal diagonal beta: 0 seconds
calculate variance: 0 seconds
optimal mixed beta (last tri

In [8]:
both_variances_lagrange(h2_jw)

	beta (diagonal):
		 0 [0.32757934 0.32757934 0.34484133]
		 1 [0.21055959 0.21055959 0.57888082]
		 2 [0.18405988 0.18405988 0.63188025]
		 3 [0.38574537 0.38574537 0.22850927]
		 4 [0.32757934 0.32757934 0.34484133]
		 5 [0.21055959 0.21055959 0.57888082]
		 6 [0.18405988 0.18405988 0.63188025]
		 7 [0.38574537 0.38574537 0.22850927]
Diagonal variance: 17.74194781113713


Trial: 
	beta (mixed):
		 0 [0.32757934 0.32757934 0.34484133]
		 1 [0.21055959 0.21055959 0.57888082]
		 2 [0.18405988 0.18405988 0.63188025]
		 3 [0.38574537 0.38574537 0.22850927]
		 4 [0.32757934 0.32757934 0.34484133]
		 5 [0.21055959 0.21055959 0.57888082]
		 6 [0.18405988 0.18405988 0.63188025]
		 7 [0.38574537 0.38574537 0.22850927]
Mixed variance: 17.457187282027824


Trial: 
	beta (mixed):
		 0 [0.32757934 0.32757934 0.34484133]
		 1 [0.21055959 0.21055959 0.57888082]
		 2 [0.18405988 0.18405988 0.63188025]
		 3 [0.38574537 0.38574537 0.22850927]
		 4 [0.32757934 0.32757934 0.34484133]
		 5 [0.21055959 0.2

In [9]:
both_variances_lagrange(lih_jw)

	beta (diagonal):
		 0 [0.37694755 0.37694755 0.2461049 ]
		 1 [0.17258567 0.17258567 0.65482865]
		 2 [0.13456529 0.13456529 0.73086942]
		 3 [0.18048016 0.18048016 0.63903969]
		 4 [0.30734537 0.30734537 0.38530925]
		 5 [0.22818266 0.22818266 0.54363468]
		 6 [0.37694755 0.37694755 0.2461049 ]
		 7 [0.17258567 0.17258567 0.65482865]
		 8 [0.13456529 0.13456529 0.73086942]
		 9 [0.18048016 0.18048016 0.63903969]
		 10 [0.30734537 0.30734537 0.38530925]
		 11 [0.22818266 0.22818266 0.54363468]
Diagonal variance: 14.792751908499572


Trial: 
	beta (mixed):
		 0 [0.37694755 0.37694755 0.2461049 ]
		 1 [0.17258567 0.17258567 0.65482865]
		 2 [0.13456529 0.13456529 0.73086942]
		 3 [0.18048016 0.18048016 0.63903969]
		 4 [0.30734537 0.30734537 0.38530925]
		 5 [0.22818266 0.22818266 0.54363468]
		 6 [0.37694755 0.37694755 0.2461049 ]
		 7 [0.17258567 0.17258567 0.65482865]
		 8 [0.13456529 0.13456529 0.73086942]
		 9 [0.18048016 0.18048016 0.63903969]
		 10 [0.30734537 0.30734537 0.385309

In [11]:
both_variances_lagrange(h2o_jw)

	beta (diagonal):
		 0 [0.43257588 0.43257588 0.13484825]
		 1 [0.24098406 0.24098406 0.51803188]
		 2 [0.13870758 0.13870758 0.72258484]
		 3 [0.13958006 0.13958006 0.72083988]
		 4 [0.16985395 0.16985395 0.6602921 ]
		 5 [0.26752985 0.26752985 0.4649403 ]
		 6 [0.11647801 0.11647801 0.76704398]
		 7 [0.43257588 0.43257588 0.13484825]
		 8 [0.24098406 0.24098406 0.51803188]
		 9 [0.13870758 0.13870758 0.72258484]
		 10 [0.13958006 0.13958006 0.72083988]
		 11 [0.16985395 0.16985395 0.6602921 ]
		 12 [0.26752985 0.26752985 0.4649403 ]
		 13 [0.11647801 0.11647801 0.76704398]
Diagonal variance: 257.5439313166082


Trial: 
	beta (mixed):
		 0 [0.43257588 0.43257588 0.13484825]
		 1 [0.24098406 0.24098406 0.51803188]
		 2 [0.13870758 0.13870758 0.72258484]
		 3 [0.13958006 0.13958006 0.72083988]
		 4 [0.16985395 0.16985395 0.6602921 ]
		 5 [0.26752985 0.26752985 0.4649403 ]
		 6 [0.11647801 0.11647801 0.76704398]
		 7 [0.43257588 0.43257588 0.13484825]
		 8 [0.24098406 0.24098406 0.518031

# Scratch

In [12]:
from var_opt import build_influential_pairs

In [43]:
def EDITED_calculate_product_term_mixed(Q, R, bitstring_HF, β):
    # assume Q,R are already admissable!
    # assume β, m are of correct structure!
    # \prod_{i | Q_i=R_i\neq I} \beta_{i,Q_i}^{-1} \prod_{i | Q_i\neq R_i} m_i
    pauli_to_index = {'X': 0, 'Y': 1, 'Z': 2}
    prodb = 1.0
    prodm = 1.0
    for i in range(len(Q)):
        if Q[i] == R[i] and Q[i] != 'I':
            qubit = (len(Q)-1)-i  # qiskit ordering
            index = pauli_to_index[Q[i]]
            b = β[qubit][index]
            prodb *= b**(-1)
        if Q[i] != R[i]:
            # then Q[i], R[i] are of the form I,Z or Z,I
            bit = int(bitstring_HF[i])
            m = (-1)**bit
            prodm *= m
    return prodb, prodm

In [44]:
ham = h2_jw
dic_tf = h2_jw.pauli_rep.dic_tf
n = ham.num_qubits
bitstring_HF = ham.read_bitstring_HF()
pairs = build_influential_pairs(dic_tf, n)

β = ham.pauli_rep.local_dists_optimal('mixed', 'lagrange', bitstring_HF=bitstring_HF)

In [49]:
Qzero = pairs[1][0]

for Q, R in pairs:
    if Q != Qzero:
        continue
    alphaQ = dic_tf[Q]
    alphaR = dic_tf[R]
    prodb, prodm = EDITED_calculate_product_term_mixed(Q, R, bitstring_HF, β)
    print("=")
    if Q==R:
        print("diagonal")
    print(alphaQ * alphaR * prodb * prodm)

=
diagonal
0.14764456566289413
=
-0.018233903938292984
=
-0.021167654224968552
=
-0.05031463308708187
=
-0.08434534174417141
=
0.06986224908905643
=
-0.032602895960064625
